In [ ]:
!python3 -m pip install --upgrade pip > /dev/null
!pip install numpy pandas seaborn > /dev/null
import numpy as np
import seaborn as sns
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Example of sampling


## Sampling from known distributions
We assume that height, weight and income follows a normal distribution. 

In the following example we assume that:

#### Distribution
* The mean hight of the population is **180 cm** and the standard deviation is **15 cm**. (Male and female combined)
* The mean weight of the population is **80 kg** and the standard deviation is **12 kg**. (Male and female combined)
* The mean income is **650000 NOK** and the standard deviation is **50000**.  

#### Correlation
* The correlation between height and weight is **0.8**
* The correlation between **income** and **height** is **0.3**
* The correlation between **income** and **weight** is **-0.2** 

#### _Feel free to tweek these parameters_

_Resources:_  
https://numpy.org/doc/stable/reference/random/generated/numpy.random.multivariate_normal.html  
https://towardsdatascience.com/multivariate-normal-distribution-562b28ec0fe0


In [ ]:
# Define statistical properties of population
mean_height = 170
sd_height = 15

mean_weight = 80
sd_weight = 12

mean_income = 650000
sd_income=50000

std_matrix = np.array([
    [sd_height, 0, 0],
    [0, sd_weight, 0],
    [0,0, sd_income]
])


# Specify desired correlation between variables
corr_height_weight = 0.8
corr_height_income = 0.3
corr_weight_income = -0.2

# Creating covariance matrix to calculate covariance between distributions.
correlation_matrix = np.array([
    [1.0, corr_height_weight, corr_height_income],
    [corr_height_weight, 1.0, corr_weight_income],
    [corr_height_income, corr_weight_income , 1.0]
])
covariance_matrix = np.dot(std_matrix, np.dot(correlation_matrix, std_matrix))


# Sampling from multivariate_distribtuion
num_samples = 10000
samples = np.random.default_rng().multivariate_normal((mean_height, mean_weight, mean_income), covariance_matrix, num_samples)

# Adding samples to Pandas DataFrame
samples_df=pd.DataFrame(samples, columns=["Height", "Weight", "Income"])


### Lets have a look at the data

In [ ]:
samples_df.corr()

In [ ]:
# looking at standard deviations.

print(f"Percentage of people taller than 160 cm: {samples_df[samples_df['Height'] > 160]['Height'].count()/num_samples*100}")
print(f"Percentage of people shorter than 200 cm: {samples_df[samples_df['Height'] < 200]['Height'].count()/num_samples*100}")
print(f"Percentage of people earning more than than 750.000 NOK: {samples_df[samples_df['Income'] > 750000]['Income'].count()/num_samples*100}")

In [ ]:
# Print actual descirptive
samples_df.describe()

In [ ]:
fig, ax =plt.subplots(1,3, figsize=(16,8))
sns.histplot(samples_df['Height'], ax=ax[0])
sns.histplot(samples_df['Weight'], ax=ax[1])
sns.histplot(samples_df['Income'], ax=ax[2])
fig.show()


### We look at the relationship between height and length

In [ ]:
sns.jointplot(data=samples_df, x="Height", y="Weight", ax=ax[0])

In [ ]:
# Tallest person
print(samples_df.iloc[samples_df["Height"].idxmax()])


In [ ]:
# Shortest person
print(samples_df.iloc[samples_df["Height"].idxmin()])

### Adding deterministic relationships

Instead of "sampling" or predicting field with deterministic to other other fields we calculate and add them!
For example BMI

In [ ]:
def calculate_bmi(x):
    return x["Weight"] / (x["Height"]/100)**2

samples_df["BMI"]=samples_df.apply(calculate_bmi,axis=1)

samples_df

# Utilize ML models

Often understanding the relationships in data can be hard to detect. Here is where machine learning shines. An example is sequentially building datasets with machine learning. 

Consider:
1. Sample from $A$ to get $A'$
2. Build a model $F1:E \sim A$
3. Synthesize $E$ as $E' = F1(A')$
4. Build a model $F2:C \sim A + E$
5. Synthesize $C$ as $C' = F2(A', E')$
6. Build a model $F3:B \sim A + E + C$
7. Synthesize $B$ as $B' = F3(A', E' +C')$
8. Build a model $F4:D \sim A + E +C + B$
9. Synthesize $D$ as $D' = F4(A', E', C' B')$

#### OBS: The method requires access to original data that models the relationship.

In this example we use cardio-vascular dataset to model gender from already sampled data. The dataset holds both **height**, **weight** and **gender**. 

_Resources:_  
https://www.kaggle.com/sulianova/cardiovascular-disease-dataset  
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html



In [ ]:
from sklearn.linear_model import LogisticRegression

# Loading dataset to create gender generator
with open("data/cardio_train.csv") as file:
    data = pd.read_csv(file, delimiter=";")
data.head()

In [ ]:
# Fitting linear regressor to predict gender from height and weight

X = data[["height", "weight"]]
y = data["gender"]
clf = LogisticRegression().fit(X, y)

In [ ]:
# Assigning gender based on height and weight

def generate_gender(x):
    categories = ["female", "male"]
    probability = clf.predict_proba([[x["Height"],x["Weight"]]])[0]
    return np.random.choice(categories, 1, p=probability)[0]

samples_df["Gender"]=samples_df.apply(generate_gender,axis=1)


### Lets look at the count of gender in the distribution

In [ ]:
sns.countplot(samples_df['Gender'])

In [ ]:
samples_df.head()

# Enter deep neural networks

When relationships are complicated we can utilize generative techniques from neural netowrks.  Examples of generative techniques are:

* **DCGAN** to generate faces: https://keras.io/examples/generative/dcgan_overriding_train_step/
* **Autoencoders**: https://www.tensorflow.org/tutorials/generative/autoencoder
* **RNN** to generate Shakespear: http://karpathy.github.io/2015/05/21/rnn-effectiveness/ 


In this example we utilize the technique desciribed in Andrej Karpathys blogpost "The Unreasonable Effectiveness of Recurrent Neural Networks". The post is refined here: 

* https://www.tensorflow.org/tutorials/text/text_generation

## The rational
We want to couple a reasonable name to each sample in our population. Let's train a neural network to create plausible names based on gender. 

To accomplish this we have scraped the 1000 most common names for men and the 1000 most common names for women in Norway. The names are denoted with a special characted in front to indicate if the name is **male** (-) or **female** (*)

In [ ]:
# Ensure that tensorflow is installed
!pip install tensorflow > /dev/null 

# Import nessesary libraries
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [ ]:
# Load "Name dataset"
with open("data/navn.txt") as file:
    navn = file.read()

# Find unique characters in text. Used to encode charactes
vocab = sorted(set(navn))

# Create functions to convert chars to ints and from ints to chars
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))
chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(navn, input_encoding='UTF-8'))
all_ids.numpy()

In [ ]:
all_chars = chars_from_ids(all_ids)
all_chars.numpy()

In [ ]:
# Create dataset from loaded text
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

seq_length = 10
examples_per_epoch = len(navn)//(seq_length+1)

# Create "sequences of text" from full text
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
dataset = sequences.map(split_input_target)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 256


model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
# Train the model

EPOS = 30 # Change to specify how many times training should occur over. 

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)
history = model.fit(dataset, epochs=EPOS)

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, next_char, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
# Creating initial state from genders in our sample
next_char=list(map(lambda x: "-" if x == "male" else "*",samples_df["Gender"].tolist()))

# Creating the prediction model. 
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, next_char, temperature=0.5)

# Running prediction on batch with size samples_size with length 15 char 

start = time.time()
states = None
next_char = tf.constant(next_char)
result = [next_char]

for n in range(15):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, '\n\n' + '_'*80)


print(f"\nRun time: {end - start}")

In [ ]:
names = list(result.numpy())
names = list(map(lambda x: x.splitlines()[0].decode("utf-8").replace("*", "").replace("-", ""), names))

In [ ]:
samples_df["Names"] = names

In [ ]:
samples_df[samples_df["Gender"]=="male"].head(20)

In [ ]:
samples_df[samples_df["Gender"]=="female"].head(20)

In [ ]:
# Compute the correlation matrix
corr = samples_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1.0, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})